# 1 Introduction

In this NoteBook, I will conduct exploratory data analysis (EDA) and clean the EdNet dataset by handling null values, parsing timestamps and eliminating duplicate records to ensure data integrity. Following this, I will perform comprehensive data processing and mining to extract meaningful patterns. A crucial step involves constructing a user-content interaction matrix, which will facilitate insightful analysis of engagement. The preprocessed data will be systematically saved in the `/data/cleaned/` directory for use. Lastly, NLP techniques will be leveraged to enhance data interpretation and extract valuable insights.

## 1.2 Importing the required libraries

I will start by loading the required libraries

In [ ]:
# Importing necessary libraries for data manipulation & visualization
import numpy as np # Is used for numerical operations and handling arrays
import pandas as pd # Is used for handling structured data in DataFrames
import matplotlib.pyplot as plt # Is used for creating static visualizations
import seaborn as sns # Provides statistical data visualization capabilities built on top of Matplotlib
import os # Helps with file & directory operations
from collections import Counter # Is useful for counting occurrences of elements in a dataset

# 2 Implementation
## 2.1 Loading the datasets

I will load the chosen datasets here. The datasets I have chosen from EdNet are **EdNet-KT1** for user interaction & performance logs, **Contents' questions.csv** for metadata used in content-based filtering and **Contents' lectures.csv** as a supplement for richer recommendations.

**EdNet-KT1** is very large, with over **784,309** files. I couldn't include and use all the files, so I wrote a script named `ednet_kt1_sampler.py` to curate this dataset. Here’s how it works: The provided code defines a function, `sample_kt1_logs`, which randomly selects and combines files from the EdNet-KT1 dataset directory into a single DataFrame. It first identifies CSV files following the naming convention (`u*.csv`), then randomly samples a specified number of user logs. Each sampled log is read into a DataFrame, with the user ID extracted from the filename and added as a column. The data is then merged into a single DataFrame and returned. Finally, the script calls this function with a sample size of 1000, saves the resulting dataset as `sampled_kt1_logs.csv`, ensuring a structured subset for further analysis. This code can be found in `../data/`.

In [ ]:
# Load datasets from ../data/
sampled_logs = pd.read_csv("../data/sampled_kt1_logs.csv")
questions = pd.read_csv("../data/questions.csv")
lectures = pd.read_csv("../data/lectures.csv") 

## 2.2 Data Exploration & Cleaning

In [2]:
# Paths
output_dir = "../data/cleaned"
os.makedirs(output_dir, exist_ok=True)

### Sampled KT1 Logs 

In [5]:
# === Inspect Sampled KT1 Logs ===
sampled_logs.head()

,timestamp,solving_id,question_id,user_answer,elapsed_time,user_id
0,1521971827112,1,q8098,b,12000,u145242
1,1521971855061,2,q8074,d,25000,u145242
2,1521971879724,3,q176,b,22000,u145242
3,1521971899710,4,q1279,c,17000,u145242
4,1521971943593,5,q2067,a,13333,u145242


In [6]:
print("\n=== KT1 Info ===")
sampled_logs.info()


=== KT1 Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117512 entries, 0 to 117511
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   timestamp     117512 non-null  int64 
 1   solving_id    117512 non-null  int64 
 2   question_id   117512 non-null  object
 3   user_answer   117502 non-null  object
 4   elapsed_time  117512 non-null  int64 
 5   user_id       117512 non-null  object
dtypes: int64(3), object(3)
memory usage: 5.4+ MB


In [7]:
print("\n=== KT1 Description ===")
sampled_logs.describe()


=== KT1 Description ===


,timestamp,solving_id,elapsed_time
count,1.175120e+05,117512.000000,1.175120e+05
mean,1.542604e+12,995.423829,2.492519e+04
std,2.104613e+10,1481.194482,2.844269e+04
min,1.492530e+12,1.000000,0.000000e+00
25%,1.526855e+12,79.000000,1.600000e+04
50%,1.548086e+12,360.000000,2.100000e+04
75%,1.561172e+12,1169.000000,2.900000e+04
max,1.575304e+12,6949.000000,5.468000e+06


In [8]:
print("\n=== KT1 Missing Values ===")
sampled_logs.isna().sum()


=== KT1 Missing Values ===


timestamp        0
solving_id       0
question_id      0
user_answer     10
elapsed_time     0
user_id          0
dtype: int64

In [9]:
print("\n=== KT1 Duplicates ===")
duplicate_rows = sampled_logs.duplicated().sum()  # Count how many rows are duplicates
print(f"Number of duplicate rows: {duplicate_rows}")


=== KT1 Duplicates ===
Number of duplicate rows: 0


In [10]:
# Drop rows where user_answer is null
sampled_logs = sampled_logs.dropna(subset=["user_answer"])

# Convert timestamp to datetime
sampled_logs["timestamp"] = pd.to_datetime(sampled_logs["timestamp"], unit='ms')

# Ensure types are consistent
sampled_logs["question_id"] = sampled_logs["question_id"].astype(str)
sampled_logs["user_id"] = sampled_logs["user_id"].astype(str)
sampled_logs["user_answer"] = sampled_logs["user_answer"].astype(str)

In [11]:
# Filter out 0 and extreme elapsed times (>5 min)
sampled_logs = sampled_logs[
    (sampled_logs["elapsed_time"] > 0) & 
    (sampled_logs["elapsed_time"] <= 300000)
]

### Questions

In [12]:
# === Inspect Questions ===
print("=== Questions Preview ===")
display(questions.head())

=== Questions Preview ===


,question_id,bundle_id,explanation_id,correct_answer,part,tags,deployed_at
0,q1,b1,e1,b,1,1;2;179;181,1558093217098
1,q2,b2,e2,a,1,15;2;182,1558093219720
2,q3,b3,e3,b,1,14;2;179;183,1558093222784
3,q4,b4,e4,b,1,9;2;179;184,1558093225357
4,q5,b5,e5,c,1,8;2;179;181,1558093228439


In [13]:
print("\n=== Questions Info ===")
questions.info()


=== Questions Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13169 entries, 0 to 13168
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13169 non-null  object
 1   bundle_id       13169 non-null  object
 2   explanation_id  13169 non-null  object
 3   correct_answer  13169 non-null  object
 4   part            13169 non-null  int64 
 5   tags            13169 non-null  object
 6   deployed_at     13169 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 720.3+ KB


In [14]:
print("\n=== Questions Description ===")
display(questions.describe(include='all'))


=== Questions Description ===


,question_id,bundle_id,explanation_id,correct_answer,part,tags,deployed_at
count,13169,13169,13169,13169,13169.000000,13169,1.316900e+04
unique,13169,9534,9534,4,NaN,1792,NaN
top,q18143,b5709,e5709,b,NaN,-1,NaN
freq,1,5,5,3624,NaN,797,NaN
mean,NaN,NaN,NaN,NaN,4.460172,NaN,1.304772e+12
std,NaN,NaN,NaN,NaN,1.620358,NaN,5.688381e+11
min,NaN,NaN,NaN,NaN,1.000000,NaN,-1.000000e+00
25%,NaN,NaN,NaN,NaN,3.000000,NaN,1.514962e+12
50%,NaN,NaN,NaN,NaN,5.000000,NaN,1.558093e+12
75%,NaN,NaN,NaN,NaN,5.000000,NaN,1.568688e+12


In [15]:
print("\n=== Questions Missing Values ===")
print(questions.isna().sum())


=== Questions Missing Values ===
question_id       0
bundle_id         0
explanation_id    0
correct_answer    0
part              0
tags              0
deployed_at       0
dtype: int64


In [16]:
print("\n=== Questions Duplicates ===")
duplicate_rows = questions.duplicated().sum()  # Count how many rows are duplicates
print(f"Number of duplicate rows: {duplicate_rows}")


=== Questions Duplicates ===
Number of duplicate rows: 0


In [17]:
# === Clean Questions ===
questions = questions.drop_duplicates()
questions = questions.dropna()
questions['deployed_at'] = pd.to_datetime(questions['deployed_at'], unit='ms')

### Lectures

In [18]:
# === Inspect Lectures ===
print("=== Lectures Preview ===")
display(lectures.head())

=== Lectures Preview ===


,lecture_id,part,tags,video_length,deployed_at
0,l520,5,142,-1,-1
1,l592,6,142,-1,-1
2,l1259,1,222,359000,1570424729123
3,l1260,1,220,487000,1570424738105
4,l1261,1,221,441000,1570424743162


In [19]:
print("\n=== Lectures Info ===")
lectures.info()


=== Lectures Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021 entries, 0 to 1020
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   lecture_id    1021 non-null   object
 1   part          1021 non-null   int64 
 2   tags          1021 non-null   int64 
 3   video_length  1021 non-null   int64 
 4   deployed_at   1021 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 40.0+ KB


In [20]:
print("\n=== Lectures Description ===")
display(lectures.describe(include='all'))


=== Lectures Description ===


,lecture_id,part,tags,video_length,deployed_at
count,1021,1021.000000,1021.000000,1.021000e+03,1.021000e+03
unique,1021,NaN,NaN,NaN,NaN
top,l843,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN
mean,NaN,1.842311,69.219393,1.472973e+05,8.735165e+11
std,NaN,2.908340,83.938264,1.830251e+05,7.804772e+11
min,NaN,-1.000000,-1.000000,-1.000000e+00,-1.000000e+00
25%,NaN,-1.000000,-1.000000,-1.000000e+00,-1.000000e+00
50%,NaN,1.000000,33.000000,1.220000e+05,1.570425e+12
75%,NaN,5.000000,120.000000,2.320000e+05,1.570426e+12


In [21]:
print("\n=== Lectures Missing Values ===")
print(lectures.isna().sum())


=== Lectures Missing Values ===
lecture_id      0
part            0
tags            0
video_length    0
deployed_at     0
dtype: int64


In [22]:
print("\n=== Lectures Duplicates ===")
duplicate_rows = lectures.duplicated().sum()  # Count how many rows are duplicates
print(f"Number of duplicate rows: {duplicate_rows}")


=== Lectures Duplicates ===
Number of duplicate rows: 0


In [ ]:
# Replace -1 with NaN
lectures[["part", "tags", "video_length", "deployed_at"]] = lectures[["part", "tags", "video_length", "deployed_at"]].replace(-1, np.nan)

# Convert deployed_at to datetime
lectures["deployed_at"] = pd.to_datetime(lectures["deployed_at"], unit="ms", errors="coerce")

In [24]:
# === Merge sampled logs with questions ===
merged = pd.merge(sampled_logs, questions, on='question_id', how='inner')

In [25]:
# === Save cleaned datasets ===
sampled_logs.to_csv("../data/cleaned/cleaned_sampled_logs.csv", index=False)
questions.to_csv("../data/cleaned/cleaned_questions.csv", index=False)
merged.to_csv("../data/cleaned/merged_cleaned_data.csv", index=False)